# HiMAP tutorial: mock community data

In [2]:
library(himap)

HiMAP v1.0 loaded.


Specify full main path and load raw FASTQ files. If ~ is not expanded, blastn breaks.

In [3]:
out_path = path.expand('~/data/zheng_2015_tutorial')

In [4]:
fastq_path = file.path(out_path, 'fastq')
fq_fwd = read_files(fastq_path, 'R1')
fq_rev = read_files(fastq_path, 'R2')

Generate output files for the pre-processing part of the pipeline:

In [5]:
sample_ids = sampleids_from_filenames(fq_fwd, separator='_')
fq_mer = file.path(out_path, 'himap_merged', paste0(sample_ids, '.fastq'))
fq_tri = file.path(out_path, 'himap_trimmed', paste0(sample_ids, '.fastq'))
fq_fil = file.path(out_path, 'himap_filtered', paste0(sample_ids, '.fastq'))

## Pre-processing
Merge reads, trim PCR primers and perform quality control and final trimming.

In [6]:
mergestats = merge_pairs(fq_fwd, fq_rev, fq_mer, verbose=T)

Loading FASTQ reads... OK.
Merging pairs... OK.
Writing output files... OK.


In [7]:
mergestats

/Users/igor/data/zheng_2015_tutorial/fastq/V3V4Rep1_R1.fastq
total       25600                                                       
low_pct_sim 1355                                                        
low_aln_len 2

Primer set used for amplifying V3-V4 region was 341F and 805R, which are labelled as 'V3-V4-2' in our reference table:

In [8]:
himap_option('blast_dbs')[6]

Primer1 Primer2 Primer1_sequence_5to3  Primer2_sequence_3to5
1 341F    805R    GACAGCCTACGGGNGGCWGCAG GACTACHVGGGTATCTAATCC
  Hypervariable_region DB                                                      
1 V3-V4-2              V3-V4_337F-805R_hang21_wrefseq_sequences_unique_variants
  table                                                     
1 V3-V4_337F-805R_hang21_wrefseq_table_unique_variants_R.txt

In [9]:
trimstats = remove_pcr_primers(fq_mer, fq_tri, region='V3-V4-2', verbose=T)

* PCR trimmer mode: region V3-V4-2 
(fwd: GACAGCCTACGGGNGGCWGCAG, rev: GGATTAGATACCCBDGTAGTC)
* loading file...OK.
* trimming...OK.
* saving output...OK.


In [10]:
trimstats

/Users/igor/data/zheng_2015_tutorial/himap_merged/V3V4Rep1.fastq
fwd_trim 24219                                                           
rev_trim 24127

Now find a length to trim all reads before passing them to DADA2 denoising. A good rule of thumb is length above which we have 99% of the reads:

In [11]:
seqlen.ft = sequence_length_table(fq_tri)
trim_length = ftquantile(seqlen.ft, 0.01)
trim_length

[1] 402

Trim reads to this length and then filter out the remaining reads with too many (> 2) expected errors:

In [12]:
filtstats = filter_and_trim(fq_tri, fq_fil, truncLen=trim_length)

In [13]:
filtstats

reads.in reads.out
V3V4Rep1.fastq 24244    22070

## DADA2 denoising
Denoise these filter and trimmed reads, then pass on the pre-trimmed reads (`fq_tri`) to retrieve back full-length sequences before we align them vs a reference database:

In [14]:
dada_result = dada_denoise(fq_fil, fq_tri, verbose=T)

* learn errors...Initializing error rates to maximum possible estimate.
Sample 1 - 22070 reads in 14060 unique sequences.
   selfConsist step 2 
   selfConsist step 3 
   selfConsist step 4 
   selfConsist step 5 
Convergence after  5  rounds.
8872140  total bases in  22070  reads used for learning the error model.
 OK.
* processing  V3V4Rep1.fastq
Sample 1 - 22070 reads in 14060 unique sequences.
Trimmed length:  402  nt.
Retrieving full-length sequences...
Sample  1 . Load...OK. Consensus...OK. Update...OK.


Extract sequence abundance from DADA2 output:

In [15]:
ab.dt = sequence_abundance(dada_result)

* generating sequence table...

The sequences being tabled vary in length.


 OK.
* removing bimeras...

Identified 28 bimeras out of 45 input sequences.


 OK.
* adding together sequences that differ in shifts on lengths...collapse:
* generating temporary files...OK.
* blast word size: 322 
* running blast...blast status:  0
OK.
* selecting ends-free alignments...OK.
* no sequences need collapsing.
* cleaning up temporary files...OK.
* returning input.
 OK.


## BLAST
Align the un-trimmed sequences to the HiMAP reference database for V3-V4 region:

In [16]:
blast_output = blast(ab.dt, region='V3-V4-2', verbose=T)

* blast input type: abundance table
* blast out: OK. blast best: OK. copy number table: OK.
merge: OK. Fix overhang differences:..OK. OK.


## OSU abundance table
From sequence abundance table and BLAST output, estimate the abundance of each Operational Strain Unit (OSU):

In [17]:
osu_ab.dt = abundance(ab.dt, blast_output)

In [18]:
osu_ab.dt

sample_id osu_id  osu_count pctsim
1  V3V4Rep1     1074 2829      100.00
2  V3V4Rep1      206 2420      100.00
3  V3V4Rep1     1093  952      100.00
4  V3V4Rep1     2440  927      100.00
5  V3V4Rep1       68  598      100.00
6  V3V4Rep1     1056  326      100.00
7  V3V4Rep1     2715  295      100.00
8  V3V4Rep1      754  273      100.00
9  V3V4Rep1     1309  248      100.00
10 V3V4Rep1     2336   95      100.00
11 V3V4Rep1     1104   54      100.00
12 V3V4Rep1      544   34      100.00
13 V3V4Rep1     2525   34      100.00
14 V3V4Rep1     1263   32      100.00
15 V3V4Rep1     1208   11      100.00
16 V3V4Rep1  1000017    6       99.29
   species                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
1  Streptococcus_mutans_[106]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
2  Escherichia_coli_[7164],Shigella_sonnei_[854],Shigella_flexneri_[67],Shigella_boydii_[32],Escherichia_fergusonii_[7],Escherichia_albertii_[2],Shigella_dysenteriae_[2],Achromobacter_sp._ATCC35328,Brenneria_alni_pvfi20,Citrobacter_braakii_SCC4,Escherichi

## Taxonomy
Assign NCBI taxonomy classification for each OSU:

In [19]:
osu_tax.dt = taxonomy(osu_ab.dt)

In [21]:
osu_tax.dt

osu_id  pctsim phylum                              
1      206 100.00 Proteobacteria_[8143]               
2  1000017  99.29 Proteobacteria_[8860],Firmicutes_[1]
3      544 100.00 Proteobacteria_[1]                  
4       68 100.00 Firmicutes_[1970]                   
5     2715 100.00 Firmicutes_[22]                     
6     1208 100.00 Actinobacteria_[151]                
7     2336 100.00 Proteobacteria_[587]                
8      754 100.00 Proteobacteria_[1886]               
9     2525 100.00 Firmicutes_[25]                     
10    1104 100.00 Firmicutes_[7]                      
11    1263 100.00 Proteobacteria_[458]                
12    1309 100.00 Firmicutes_[7827]                   
13    2440 100.00 Proteobacteria_[14]                 
14    1093 100.00 Firmicutes_[3]                      
15    1074 100.00 Firmicutes_[106]                    
16    1056 100.00 Firmicutes_[490]                    
   class                                                        
1  Gammaproteobacteria_[8142],Betaproteobacteria_[1]            
2  Gammaproteobacteria_[8859],Betaproteobacteria_[1],Bacilli_[1]
3  Gammaproteobacteria_[1]                                      
4  Bacilli_[1970]                                               
5  Clostridia_[22]                                              
6  Actinobacteria_[151]                                         
7  Epsilonproteobacteria_[587]                                  
8  Gammaproteobacteria_[1886]                                   
9  Bacilli_[25]                                                 
10 Bacilli_[7]                                                  
11 Betaproteobacteria_[457],Gammaproteobacteria_[1]             
12 Bacilli_[7827]                                               
13 Alphaproteobacteria_[14]                                     
14 Bacilli_[3]                                                  
15 Bacilli_[106]                                                
16 Bacilli_[490]                                                
   order                                                                              
1  Enterobacterales_[8142],Burkholderiales_[1]                                        
2  Enterobacterales_[8857],Pseudomonadales_[2],Burkholderiales_[1],Lactobacillales_[1]
3  Pseudomonadales_[1]                                                                
4  Bacillales_[1970]                                                                  
5  Clostridiales_[22]                                                                 
6  Propionibacteriales_[151]                                                          
7  Campylobacterales_[587]                                                            
8  Pseudomonadales_[1886]                                                             
9  Lactobacillales_[25]                                                               
10 Bacillales_[7]                                                                     
11 Neisseriales_[457],Pseudomonadales_[1]                                             
12 Bacillales_[7826],Lactobacillales_[1]                                              
13 Rhodobacterales_[14]                                                               
14 Bacillales_[3]                                                                     
15 Lactobacillales_[106]                                                              
16 Lactobacillales_[490]                                                              
   family                                                                                                   
1  Enterobacteriaceae_[8141],Alcaligenaceae_[1],Pectobacteriaceae_[1]                                       
2  Enterobacteriaceae_[8856],Moraxellaceae_[2],Alcaligenaceae_[1],Pectobacteriaceae_[1],Lactobacillaceae_[1]
3  Moraxellaceae_[1]                                                                                        
4  Bacillaceae_[1970]                           

In [25]:
blast_output$alignments[qseqid==17]

variant_id qseqid qlen length qstart qend sstart send slen score match
1    00005      17     427  421    7      427  27     447  470  2091  419  
2    00005      17     427  421    7      427  27     447  470  2091  419  
3    00005      17     427  421    7      427  27     447  470  2091  419  
4    00005      17     427  421    7      427  27     447  470  2091  419  
5    00005      17     427  421    7      427  27     447  470  2091  419  
6    00005      17     427  421    7      427  27     447  470  2091  419  
7    00005      17     427  421    7      427  27     447  470  2091  419  
8    00005      17     427  421    7      427  27     447  470  2091  419  
9    00005      17     427  421    7      427  27     447  470  2091  419  
10   00005      17     427  421    7      427  27     447  470  2091  419  
11   00005      17     427  421    7      427  27     447  470  2091  419  
12   00005      17     427  421    7      427  27     447  470  2091  419  
13   00005      17     427  421    7      427  27     447  470  2091  419  
14   00005      17     427  421    7      427  27     447  470  2091  419  
15   00005      17     427  421    7      427  27     447  470  2091  419  
16   00005      17     427  421    7      427  27     447  470  2091  419  
17   00005      17     427  421    7      427  27     447  470  2091  419  
18   00005      17     427  421    7      427  27     447  470  2091  419  
19   00005      17     427  421    7      427  27     447  470  2091  419  
20   00005      17     427  421    7      427  27     447  470  2091  419  
21   00005      17     427  421    7      427  27     447  470  2091  419  
22   00005      17     427  421    7      427  27     447  470  2091  419  
23   00005      17     427  421    7      427  27     447  470  2091  419  
24   00005      17     427  421    7      427  27     447  470  2091  419  
25   00005      17     427  421    7      427  27     447  470  2091  419  
26   00005      17     427  421    7      427  27     447  470  2091  419  
27   00005      17     427  421    7      427  27     447  470  2091  419  
28   00005      17     427  421    7      427  27     447  470  2091  419  
29   00005      17     427  421    7      427  27     447  470  2091  419  
30   00005      17     427  421    7      427  27     447  470  2091  419  
⋮    ⋮          ⋮      ⋮    ⋮      ⋮      ⋮    ⋮      ⋮    ⋮    ⋮     ⋮    
8865 05585      17     427  421    7      427  27     447  470  2091  419  
8866 05677      17     427  421    7      427  27     447  460  2091  419  
8867 05679      17     427  421    7      427  27     447  456  2091  419  
8868 05679      17     427  421    7      427  27     447  456  2091  419  
8869 05679      17     427  421    7      427  27     447  456  2091  419  
8870 05679      17     427  421    7      427  27     447  456  2091  419  
8871 06976      17     427  421    7      427  27     447  470  2091  419  
8872 06976      17     427  421    7      427  27     447  470  2091  419  
8873 06978      17     427  421    7      427  27     447  470  2091  419  
8874 07935      17     427  421    7      427  27     447  463  2091  419  
8875 07935      17     427  421    7      427  27     447  463  2091  419  
8876 07935      17     427  421    7      427  27     447  463  2091  419  
8877 07936      17     427  421    7      427  27     447  462  2091  419  
8878 07936      17     427  421    7      427  27     447  462  2091  419  
8879 07936      17     427  421    7      427  27     447  462  2091  419  
8880 08529      17     427  421    7      427  27     447  470  2091  419  
8881 09455      17     427  421    7      427  27     447  458  2091  419  
8882 11363      17     427  421    7      427  27     447  459  2091  419  
8883 11518      17     427  421    7      427  27     447  470  2091  419  
8884 11518      17     427  421    7      427  27     447  470  2091  419  
8885 11518      17     427  421    7      427  27    